In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from shapely.geometry import Polygon, Point
from geopy.distance import great_circle
from geopandas import GeoDataFrame
from descartes import PolygonPatch
#/Notes
#Commented out for reasons

In [29]:
#/Original CSV file
Schools = pd.read_csv("/Users/dkjayco/Desktop/Mapping/Illinois Schools and Energy CSVs/Illinois Schools and Energy CSV - 5.csv")
Schools.head()

#/Shape file
Illinois = GeoDataFrame.from_file("/Users/dkjayco/Desktop/Mapping/IL_BNDY_County/IL_BNDY_County_Ln.shp")

#/Extracting the coordinate reference system (CRS) from Illinois...
original_crs = Illinois.crs

#/Creating the target CRS
    #/Boeing used Albers' Projection for his map, where he specified two parallels and a center as the reference for 
    #/his map because area was accurate, but shape was not. Would it matter what projection meathod we used since our 
    #/framefor the map is just Illinois?...
#target_crs = {'datum':'WGS84', 'no_defs':True, 'proj':'aea', 'lat_1':35, 'lat_2':55, 'lat_0':45, 'lon_0':10}
target_crs = {'datum':'WGS84', 'no_defs':True, 'proj':'aea', 'lat_1':43, 'lat_2':36, 'lat_0':40, 'lon_0':-90}

#/...and setting my map to that reference
Illinois.to_crs(crs=target_crs, inplace=True)

In [30]:
#fig = plt.figure()
#ax = Illinois.plot()
#fig.add_axes(ax)
#plt.show()
title_font = fm.FontProperties(family='Bitstream Vera Sans', style='normal', size=15, weight='normal', stretch='normal')
annotation_font = fm.FontProperties(family='Bitstream Vera Sans', style='normal', size=10, weight='normal', stretch='normal')
backgroundcolor = '#e4f4ff'

In [37]:
# create a geometry column in our point data set for geopandas to use
Schools['geometry'] = None
for i, row in Schools.iterrows():
    Schools['geometry'][i] = Point(row['longitude'], row['latitude'])

# create a new geopandas geodataframe from the point data 
points = GeoDataFrame(Schools)

# you must specify its original CRS to convert it to a different (projected) one later
points.crs = original_crs

points

/usr/local/lib/python3.4/site-packages/IPython/kernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


latitude  longitude                             School  \
0    40.064489 -91.028207                Central High School   
1    39.816836 -91.250042         Payson-Seymour High School   
2    39.914466 -91.399203      Quincy Notre Dame High School   
3    38.901171 -89.398924             Greenville High School   
4    39.981571 -90.752685           Brown County High School   
5    42.095750 -90.133428            West Caroll High School   
6    39.892221 -90.012724     A-C Central Junior High School   
7    40.105903 -88.282347             Centennial High School   
8    40.105903 -88.282347   Champaign Centennial High School   
9    40.116583 -88.253415      Champaign Central High School   
10   38.663366 -88.483182                  Flora High School   
11   38.520145 -89.103544              Centralia High School   
12   38.619424 -89.373874                Carlyle High School   
13   38.611614 -89.558943      Central Community High School   
14   38.569114 -89.686805                Wesclin High School   
15   39.683911 -88.316485                 Arcola High School   
16   39.485302 -88.164507             Charleston High School   
17   39.473654 -88.385998                Mattoon High School   
18   41.918408 -87.645845           Lincoln Park High School   
19   41.782192 -87.671919  Lindblom Math and Science Academy   
20   41.981241 -87.707671     Northside College Preparatory    
21   40.246956 -88.646133             Blue Ridge High School   
22   40.145912 -88.978575                Clinton High School   
23   39.806384 -88.464850         Atwood Hammond High School   
24   39.868602 -88.161529            Villa Grove High School   
25   41.863794 -87.982177            Willowbrook High School   
26   39.116890 -88.558525              Effingham High School   
27   39.117500 -88.554708       Effingham Junior High School   
28   38.991810 -88.955815       Brownston Jr/Sr. High School   
29   40.460789 -88.107379    Paxton-buckley-loda High School   
..         ...        ...                                ...   
121        NaN        NaN                                NaN   
122        NaN        NaN                                NaN   
123        NaN        NaN                                NaN   
124        NaN        NaN                                NaN   
125        NaN        NaN                                NaN   
126        NaN        NaN                                NaN   
127        NaN        NaN                                NaN   
128        NaN        NaN                                NaN   
129        NaN        NaN                                NaN   
130        NaN        NaN                                NaN   
131        NaN        NaN                                NaN   
132        NaN        NaN                                NaN   
133        NaN        NaN                                NaN   
134        NaN        NaN                                NaN   
135        NaN        NaN                                NaN   
136        NaN        NaN                                NaN   
137        NaN        NaN                                NaN   
138        NaN        NaN                                NaN   
139        NaN        NaN                                NaN   
140        NaN        NaN                                NaN   
141        NaN        NaN                                NaN   
142        NaN        NaN                                NaN   
143        NaN        NaN                                NaN   
144        NaN        NaN                                NaN   
145        NaN        NaN                                NaN   
146        NaN        NaN                                NaN   
147        NaN        NaN                                NaN   
148        NaN        NaN                                NaN   
149        NaN        NaN                                NaN   
150        NaN        NaN                                NaN   

                                     

In [32]:
# calculate some margin so our data doesn't go right up to the edges of the plotting figure
x_margin_width = (points.bounds['maxx'].max() - points.bounds['minx'].min()) / 10
y_margin_width = (points.bounds['maxy'].max() - points.bounds['miny'].min()) / 3

# define the coordinates at the extent of our projected point data
xlim = (points.bounds['minx'].min() - x_margin_width, points.bounds['maxx'].max() + x_margin_width)
ylim = (points.bounds['miny'].min() - y_margin_width, points.bounds['maxy'].max() + y_margin_width)

# create a rectangle from these coordinates
spatial_extent = Polygon([(xlim[0], ylim[0]), 
                       (xlim[0], ylim[1]), 
                       (xlim[1], ylim[1]),
                       (xlim[1], ylim[0])])

In [33]:
# select the county shapes within the spatial extents of our point data
IL = Illinois[Illinois['geometry'].intersects(spatial_extent)]

# set dimensions in inches for the plotting figure size
xdim = (xlim[1] - xlim[0]) / 400000
ydim = (ylim[1] - ylim[0]) / 400000

In [34]:
def get_patches(inspected_county):
    
    #/ specify the colors for our map
    facecolor = '#f7f7f7'
    inspected_facecolor = '#eeeeee'
    edgecolor = '#cccccc'
    
    #/ create a list to contain a descartes PolygonPatch object for each Polygon in the GeoDataFrame geometry column
    patches = []

    for i, row in IL.iterrows():

        #/ if this row contains a Polygon object
        #if 
        type(row['geometry']) == Polygon
        fc = inspected_facecolor if row['NAME'] in inspected_county else facecolor
        patch = PolygonPatch(row['geometry'], fc=fc, ec=edgecolor, zorder=0)
        patches.append(patch)

        #/ else, this row contains a MultiPolygon object - this is a shapely object that contains multiple Polygon objects
        #/ for example, countries that contain islands will have one Polygon shape for their mainland, and others for the island geometries
        #else:
            #/ for each Polygon object in the MultiPolygon
            #for polygon in row['geometry']:
                #fc = visited_facecolor if row['NAME'] in visited_countries else facecolor
                #patch = PolygonPatch(polygon, fc=fc, ec=edgecolor, zorder=0)
                #patches.append(patch)
    return patches                

In [35]:
# get a list of inspected counties so we can shade those patches a different color
inspected_county = Schools['County'].unique()
#county = 

In [39]:
# create a figure, axis, and set the background color
fig = plt.figure(figsize=(xdim, ydim))
ax = fig.add_subplot(111)
ax.set_axis_bgcolor(backgroundcolor)

# add each patch we extracted from the GeoDataFrame's geometry column to the axis
for patch in get_patches(inspected_county):
    ax.add_patch(patch)

# add the projected point data to the axis as a scatter plot
points_scatter = ax.scatter(x=points["x"], y=points['y'], c='m', alpha=0.4, s=100)

ax.set_title('Projected shapefile and GPS coordinates', fontproperties=title_font)

# set the axes limits
ax.set_xlim(xlim)
ax.set_ylim(ylim)

# remove the tickmarks as these are projected geometries, the ticks are confusing northings/eastings
ax.set_xticks([])
ax.set_yticks([])

plt.show()

KeyError: 'x'

In [ ]:
#/Rearrange the data so that it is sorted by county
#School = pd.read_csv("/Users/dkjayco/Downloads/Git/RDCEP-summer-scholars-2015/Danica's testing things.../Illinois Schools and Energy CSVs/Illinois Schools and Energy CSV - 2.csv", usecols=['School','Address','County','Biology','Chemistry','Physics','Environmental Science','Engineering','Latitude','Longitude','Accuracy'])
#School = pd.read_csv("/Users/dkjayco/Desktop/Mapping/Illinois Schools and Energy CSVs/Illinois Schools and Energy CSV - 5.csv", usecols=[0,1,2,3,4,5,6,7,8,9,10], index_col="County")
#School.head()
